In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1"
import pandas as pd
import pickle
from joblib import Parallel, delayed
import numpy as np
import time
import jax
from jax import devices
import jax.numpy as jnp
import jax.random as jr
from scipy.stats import zscore
import jax.numpy as jnp
import jax.random as jr
from collections import defaultdict
import gc
from dynamax.hidden_markov_model import LinearAutoregressiveHMM, PoissonHMM
from dynamax.hidden_markov_model import PoissonHMM 

# Get my functions
from functions import idxs_from_files, cross_validate_poismodel, cross_validate_armodel, compute_inputs


ERROR:2026-02-10 17:31:32,668:jax._src.xla_bridge:444: Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/jax/_src/xla_bridge.py", line 442, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 324, in initialize
    _check_cuda_versions(raise_on_first_error=True)
  File "/home/ines/miniconda3/envs/iblenv/lib/python3.10/site-packages/jax_plugins/xla_cuda12/__init__.py", line 281, in _check_cuda_versions
    local_device_count = cuda_versions.cuda_device_count()
RuntimeError: jaxlib/cuda/versions_helpers.cc:113: operation cuInit(0) failed: CUDA_ERROR_NO_DEVICE


In [2]:
"""" GRID SEARCH FUNCTIONS """

# Function to perform grid search for a single session
def grid_search_versatile(id, var_interest, zsc, fixed_states, truncate,
                          params, sticky, save_path, data_path, num_train_batches, method, fit_method):
    print(params)
    print(jax.devices())
    # Time job
    start = time.time()
    
    # Load mouse/session data
    mouse_name, session = id
    filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
    original_design_matrix = pd.read_parquet(filename)

    # Zscore if needed (paws)
    if zsc == True:
      array_matrix = zscore(np.array(original_design_matrix), axis=0, nan_policy='omit')
    else:
      array_matrix = np.array(original_design_matrix)

    # Remove NaNs
    filtered_matrix = array_matrix[~np.isnan(array_matrix).any(axis=1)]
    keys = np.where(original_design_matrix.keys().isin(var_interest))
    if truncate:
        design_matrix = filtered_matrix[:100000, keys]  # Truncate session
    else:
        design_matrix = filtered_matrix[:, keys] 
    
    design_matrix = np.reshape(design_matrix,(np.shape(design_matrix)[0], np.shape(design_matrix)[2]))

    fit_id = str(mouse_name + session)

    print(f"Fitting session {fit_id}...")

    num_timesteps = np.shape(design_matrix)[0]
    emission_dim = np.shape(design_matrix)[1]
    shortened_array = np.array(design_matrix[:(num_timesteps // num_train_batches) * num_train_batches])
    train_emissions = jnp.stack(jnp.split(shortened_array, num_train_batches))

    # Initialize result containers
    all_lls, all_baseline_lls = defaultdict(dict), defaultdict(dict)
    all_init_params, all_fit_params = defaultdict(dict), defaultdict(dict)

    if var_interest == ['Lick count']:
        num_states, _, kappas = params
        if fixed_states:
            # Grid search
            for kappa in kappas:
                test_poishmm = PoissonHMM(num_states, emission_dim, transition_matrix_stickiness=kappa)
                all_val_lls, fit_params, init_params, baseline_lls = cross_validate_poismodel(test_poishmm, jr.PRNGKey(0),
                                                                                            train_emissions, num_train_batches, fit_method, num_iters=100)

                all_lls[kappa] = all_val_lls
                all_baseline_lls[kappa] = baseline_lls
                all_init_params[kappa] = init_params
                all_fit_params[kappa] = fit_params
        else:
            for state in num_states:
                # Grid search
                for kappa in kappas:
                    test_poishmm = PoissonHMM(state, emission_dim, transition_matrix_stickiness=kappa)
                    all_val_lls, fit_params, init_params, baseline_lls = cross_validate_poismodel(test_poishmm, jr.PRNGKey(0),
                                                                                               train_emissions, num_train_batches, fit_method, num_iters=100)

                    all_lls[state][kappa] = all_val_lls
                    all_baseline_lls[state][kappa] = baseline_lls
                    all_init_params[state][kappa] = init_params
                    all_fit_params[state][kappa] = fit_params

    else:
        num_states, all_num_lags, kappas = params
        if fixed_states:
            # Grid search
            for lag in all_num_lags:
                for kappa in kappas:
                    test_arhmm = LinearAutoregressiveHMM(num_states, emission_dim, num_lags=lag, transition_matrix_stickiness=kappa)
                    my_inputs = compute_inputs(shortened_array, lag, emission_dim)
                    train_inputs = jnp.stack(jnp.split(my_inputs, num_train_batches))

                    all_val_lls, fit_params, init_params, baseline_lls = cross_validate_armodel(
                        test_arhmm, jr.PRNGKey(0), train_emissions, train_inputs, method, num_train_batches, fit_method
                    )

                    all_lls[lag][kappa] = all_val_lls
                    all_baseline_lls[lag][kappa] = baseline_lls
                    all_init_params[lag][kappa] = init_params
                    all_fit_params[lag][kappa] = fit_params
        else:
            # Grid search
            for state in num_states:
                for lag in all_num_lags:
                    for kappa in kappas:
                        test_arhmm = LinearAutoregressiveHMM(state, emission_dim, num_lags=lag, transition_matrix_stickiness=kappa)
                        my_inputs = compute_inputs(shortened_array, lag, emission_dim)
                        train_inputs = jnp.stack(jnp.split(my_inputs, num_train_batches))

                        all_val_lls, fit_params, init_params, baseline_lls = cross_validate_armodel(
                            test_arhmm, jr.PRNGKey(0), train_emissions, train_inputs, method, num_train_batches, fit_method
                        )

                        all_lls[state][lag][kappa] = all_val_lls
                        all_baseline_lls[state][lag][kappa] = baseline_lls
                        all_init_params[state][lag][kappa] = init_params
                        all_fit_params[state][lag][kappa] = fit_params

    # Save results
    mouse_results = all_lls, all_baseline_lls, all_init_params, all_fit_params, design_matrix, params
    result_filename = os.path.join(save_path, f"{'best_sticky' if sticky else 'best'}_results_{var_interest[0]}_{fit_id}")
    with open(result_filename, "wb") as f:
        pickle.dump(mouse_results, f)

    print(f"Session {fit_id} completed.")
    print("Time:", time.time() - start)
    del mouse_results, all_lls, all_baseline_lls, all_init_params, all_fit_params
    gc.collect()


# Main function for parallel processing
def run_grid_search_parallel(idxs, var_interest, zsc, fixed_states, truncate, params, sticky,
                             save_path, data_path, num_train_batches, method, fit_method, n_jobs):

    # Create folder where to dump data
    if not os.path.exists(save_path):
        # Create a new directory because it does not exist
        os.makedirs(save_path)
   
    # Identify sessions to process
    sessions_to_process = []
    for m, mat in enumerate(idxs):
        mouse_name = mat[37:]
        session = mat[:36]
        fit_id = str(mouse_name + session)
        result_filename = os.path.join(save_path, f"{'best_sticky' if sticky else 'best'}_results_{var_interest[0]}_{fit_id}")
        if not os.path.exists(result_filename):
            sessions_to_process.append((mouse_name, session))
    # sessions_to_process = sessions_to_process

    print(f"Found {len(sessions_to_process)} sessions to process.")

    # Run grid search in parallel
    Parallel(n_jobs=n_jobs)(
        delayed(grid_search_versatile)(
            id, var_interest, zsc,
            fixed_states, truncate, params, sticky, save_path, data_path, num_train_batches, method, fit_method
        ) for id in sessions_to_process
    )

# Main function for parallel processing
def run_grid_search_serial(idxs, var_interest, zsc, fixed_states, truncate, params, sticky,
                             save_path, data_path, num_train_batches, method, fit_method):

    # Create folder where to dump data
    if not os.path.exists(save_path):
        # Create a new directory because it does not exist
        os.makedirs(save_path)
   
    # Identify sessions to process
    sessions_to_process = []
    for m, mat in enumerate(idxs):
        mouse_name = mat[37:]
        session = mat[:36]
        fit_id = str(mouse_name + session)
        result_filename = os.path.join(save_path, f"{'best_sticky' if sticky else 'best'}_results_{var_interest[0]}_{fit_id}")
        if not os.path.exists(result_filename):
            sessions_to_process.append((mouse_name, session))
    # sessions_to_process = sessions_to_process

    print(f"Found {len(sessions_to_process)} sessions to process.")
    
    # Run grid search in series
    for m, mat in enumerate(sessions_to_process):

        grid_search_versatile(
            mat, var_interest, zsc,
            fixed_states, truncate, params, sticky, save_path, data_path, num_train_batches, method, fit_method
        )

In [3]:
kcenia_sessions = ['d7bf2611-00bc-42d3-95c6-ac120753cd94',
                   '72a5e765-efaf-49a5-831b-3cc35ec76517',
                    '933765b5-1f4e-4877-b889-7cc87f6ddb64',
                    'a196fce9-7eae-4c11-ae01-25a9aa59b593',
                    'd7bf2611-00bc-42d3-95c6-ac120753cd94',
                    'e69221c7-b533-42dc-8631-000279a45a70',
                    '364d72f6-483c-4146-88df-e77e86f22a60',
                    '89a9b132-7cda-4d97-8d28-448119c0910e',
                    'a3657685-f782-41e6-bc94-2fb1061b1fec',
                    'acdb349a-0181-4455-82c1-2cbdc24ca362',
                    'b27c0221-781d-4519-bd14-9a03938edc12',
                    'e505245e-585c-451e-b590-60f3ef714a89',
                    'f9b1dafa-1ec0-46e2-a72c-987535b72bb6',
                    '04482c20-f984-4d90-97d6-6791bae1e6f7',
                    '291e25f7-b35f-4d11-815f-1429127059d4',
                    '5853e474-2c8f-448f-8ef8-c753eaceb408',
                    '604dc7eb-533f-4d32-a1a3-064f7144656e',
                    '69544b1b-7788-4b41-8cad-2d56d5958526',
                    '89b8ef70-e620-49c2-a0f7-09890ba9fc0e',
                    '941df82c-9894-4280-bf3c-09323658b299',
                    '9ea8fa32-3cbf-4ea1-a341-285837caa137',
                    'bd5dbb12-dbfd-488c-b5d9-2b06bed078dd',
                    'd78d8da7-980d-4550-a039-8217abe51d9a',
                    'f5369fa4-4bc2-4c65-9217-bea27df0f9ca',
                    '3bef683f-46a4-4b98-8b53-8eb0d5bdf82b',
                    '475d6c49-1e0f-49db-b591-645c3283926e',
                    '8c2639e9-ad67-4c07-bdc0-4a5ba994375c',
                    'a06c142a-9b90-4639-a66e-0ed464eeb476',
                    'd3b1eae9-7b19-4d09-9560-23092cce572c',
                    'd8b48595-6aa5-4e4b-b457-13b1b936248c',
                    'f0cfd8b5-4568-4944-96c9-23d75aed659c',
                    'f2343157-4824-4fa2-84c1-a8bfebe21a8f',
                    'fa9efd38-29f9-41a6-a005-21c9fc4be5dc',
                    '404d1b7e-f6f2-4122-bd2d-9caa6a0e7777',
                    '0de78321-9dce-4964-af4b-d0f00a41c647',
                    '929662b6-4182-4ee7-8a50-ec788520c9e2',
                    '68f894bf-2268-4773-aa8d-ac8ab168320e',
                    'dd549116-68a1-462d-90ef-f4e0ec5c69fc',
                    '6f224adf-e791-4255-b6ab-e4e1f059813a',
                    'e470c52d-25bc-4dac-9679-2dc0b7a3677f',
                    '1f959c77-862c-4da1-a454-74a6e99ff33e',
                    '12f1d635-8a2a-49fe-ad4d-efa33b89301b',
                    '163fa79b-9b33-43ee-8d58-335f4c12d370']

In [4]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""""""""""""""""""""""""""""" 'RUN CODE' """"""""""""""""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


prefix = '/home/ines/repositories/representation_learning_variability/'

# Parameters
n_jobs = 2  # Number of CPU cores to use

# LOAD DATA
data_path = prefix + 'paper-individuality/data/design_matrices/kcenia/'
all_files = os.listdir(data_path)
design_matrices = [item for item in all_files if 'design_matrix' in item and 'standardized' not in item]

filtered = [s for s in design_matrices if any(k in s for k in kcenia_sessions)]

idxs, mouse_names = idxs_from_files(filtered)

var_interest = ['whisker_me']
var_interest = ['Lick count']
# var_interest = ['avg_wheel_vel']

""" GRID """
all_num_lags = [1, 3, 5, 7, 10, 15, 20, 30]
all_num_lags=list(range(1, 20, 2))
kappas=[0, 0.5, 1, 2, 3, 4, 5, 10, 20, 100]

fixed_states = True
num_states = 2

if var_interest == ['whisker_me']:
    n_jobs = 1
    all_num_lags=[1, 10, 20, 40, 60] 
    all_num_lags=[1, 10, 20, 30] 
    # all_num_lags=list(range(1, 60, 10))
    kappas=[0, 5, 50]
    num_states = 2
elif var_interest == ['Lick count']:
    n_jobs = 3
    kappas=[0, 1, 5, 10, 50, 100, 500, 1000]
    num_states = 2
elif var_interest == ['avg_wheel_vel']:
    n_jobs = 1
    all_num_lags=[1, 2, 3, 4] 
    # all_num_lags=list(range(1, 60, 10))
    kappas=[0, 5, 50]
    

params = num_states, all_num_lags, kappas

""" FITTING PARAMETERS """
num_train_batches = 5
method='prior'
fit_method='em'
zsc = False
truncate = False

if var_interest == ['whisker_me']:
    zsc = True
elif var_interest == ['Lick count']:
    zsc = False
elif var_interest == ['avg_wheel_vel']:
    zsc = False

fitting_params = str(num_train_batches)+'_'+method+'_'+fit_method+'_zsc_'+str(zsc)+'/'
if truncate:
    fitting_params = str(num_train_batches)+'_'+method+'_'+fit_method+'_zsc_'+str(zsc)+'_truncated/'

save_path = prefix + 'paper-individuality/data/hmm/grid_search/'+fitting_params

# # Run the grid search
# run_grid_search_parallel(
#     idxs, var_interest, zsc,
#     fixed_states=fixed_states, truncate=truncate, params=params, sticky=False,
#     save_path=save_path,  data_path=data_path, num_train_batches=num_train_batches, method=method, fit_method=fit_method, n_jobs=n_jobs)

run_grid_search_serial(
    idxs, var_interest, zsc,
    fixed_states=fixed_states, truncate=truncate, params=params, sticky=False,
    save_path=save_path,  data_path=data_path, num_train_batches=num_train_batches, method=method, fit_method=fit_method)

Found 9 sessions to process.
(2, [1, 3, 5, 7, 9, 11, 13, 15, 17, 19], [0, 1, 5, 10, 50, 100, 500, 1000])
[CpuDevice(id=0)]
Fitting session ZFM-08751364d72f6-483c-4146-88df-e77e86f22a60...
Session ZFM-08751364d72f6-483c-4146-88df-e77e86f22a60 completed.
Time: 28.442047834396362
(2, [1, 3, 5, 7, 9, 11, 13, 15, 17, 19], [0, 1, 5, 10, 50, 100, 500, 1000])
[CpuDevice(id=0)]
Fitting session ZFM-08652475d6c49-1e0f-49db-b591-645c3283926e...
Session ZFM-08652475d6c49-1e0f-49db-b591-645c3283926e completed.
Time: 33.68816685676575
(2, [1, 3, 5, 7, 9, 11, 13, 15, 17, 19], [0, 1, 5, 10, 50, 100, 500, 1000])
[CpuDevice(id=0)]
Fitting session ZFM-08751acdb349a-0181-4455-82c1-2cbdc24ca362...
Session ZFM-08751acdb349a-0181-4455-82c1-2cbdc24ca362 completed.
Time: 6.681607723236084
(2, [1, 3, 5, 7, 9, 11, 13, 15, 17, 19], [0, 1, 5, 10, 50, 100, 500, 1000])
[CpuDevice(id=0)]
Fitting session ZFM-045340de78321-9dce-4964-af4b-d0f00a41c647...
Session ZFM-045340de78321-9dce-4964-af4b-d0f00a41c647 completed.
Ti